# **Tutorial 1: Creating DataArrays and Datasets to Assess Global Climate Data**


**Week 1, Day 1, Climate System Overview**

**Content creators:** Sloane Garelick, Julia Kent

**Content reviewers:** Yosmely Bermúdez, Katrina Dobson, Younkap Nina Duplex, Danika Gupta, Maria Gonzalez, Will Gregory, Nahid Hasan, Sherry Mi, Beatriz Cosenza Muralles, Jenna Pearson, Chi Zhang, Ohad Zivan 

**Content editors:** Jenna Pearson, Chi Zhang, Ohad Zivan 

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS

### **Code and Data Sources**

Code and data for this tutorial is based on existing content from [Project Pythia](https://foundations.projectpythia.org/core/xarray/xarray-intro.html).

# **Tutorial Objectives**


As you just learned in the Introduction to Climate video, variations in global climate involve various forcings, feedbacks, and interactions between multiple processes and systems. Because of this complexity, global climate datasets are often very large with multiple dimensions and variables.

One useful computational tool for organizing, analyzing and interpreting large global datasets is XArray, an open source project and Python package that makes working with labelled multi-dimensional arrays simple and efficient.

In this first tutorial, we will use the `DataArray` and `Dataset` objects, which are used to represent and manipulate spatial data, to practice organizing large global climate datasets and to understand variations in Earth's climate system.

# Setup

Similar to `numpy`, `np`; `pandas`, `pd`; you may often encounter `xarray` imported within a shortened namespace as `xr`. `pythia_datasets` provides example data for us to work with.

In [1]:
# !pip install datetime
# !pip install numpy
# !pip install pandas
# !pip install xarray
# !pip install pythia_datasets

In [2]:
# imports
from datetime import timedelta
import numpy as np
import pandas as pd
import xarray as xr
from pythia_datasets import DATASETS
import matplotlib.pyplot as plt

In [3]:
# @title Figure Settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")

In [4]:
# @title Video 1: Video Title
#Tech team will add code to format and display the video

# Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

# Section 1: Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

## Section 1.1: Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [5]:
rand_data = 283 + 5 * np.random.randn(5, 3, 4)
rand_data

array([[[290.61645854, 281.28271168, 281.84226324, 278.61198526],
        [287.41265079, 284.35126492, 291.37705225, 282.52505946],
        [285.21029519, 282.25033567, 284.95447723, 286.34831043]],

       [[283.26057525, 289.01248113, 275.0626402 , 275.4539616 ],
        [284.68249285, 280.94575736, 280.49077327, 278.68237371],
        [282.10345323, 282.13403979, 275.69059717, 276.29680092]],

       [[283.45758266, 278.64363301, 277.70261278, 286.66049153],
        [286.64976732, 281.79493996, 293.54110883, 281.47250044],
        [287.67739165, 285.81404815, 290.57404849, 286.02542803]],

       [[277.12882683, 285.90567904, 290.64640203, 285.88070743],
        [283.22915585, 285.61280549, 276.44904728, 282.03546889],
        [278.45403384, 281.45890968, 288.01316258, 279.06508788]],

       [[287.02633878, 277.1492867 , 281.77224677, 277.13671493],
        [275.67401883, 283.77637672, 272.42099945, 281.2700804 ],
        [289.49935391, 271.0793694 , 275.71341183, 274.84101139]]])

## Section 1.2: Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [6]:
tempature = xr.DataArray(rand_data)
tempature

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[290.61645854, 281.28271168, 281.84226324, 278.61198526],
        [287.41265079, 284.35126492, 291.37705225, 282.52505946],
        [285.21029519, 282.25033567, 284.95447723, 286.34831043]],

       [[283.26057525, 289.01248113, 275.0626402 , 275.4539616 ],
        [284.68249285, 280.94575736, 280.49077327, 278.68237371],
        [282.10345323, 282.13403979, 275.69059717, 276.29680092]],

       [[283.45758266, 278.64363301, 277.70261278, 286.66049153],
        [286.64976732, 281.79493996, 293.54110883, 281.47250044],
        [287.67739165, 285.81404815, 290.57404849, 286.02542803]],

       [[277.12882683, 285.90567904, 290.64640203, 285.88070743],
        [283.22915585, 285.61280549, 276.44904728, 282.03546889],
        [278.45403384, 281.45890968, 288.01316258, 279.06508788]],

       [[287.02633878, 277.1492867 , 281.77224677, 277.13671493],
        [275.67401883, 283.77637672, 272.42099945, 281.2700804 ],
        [289.49935391, 271.0793694 , 275.71341183, 274.84101139]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

## Section 1.3: Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [7]:
tempature = xr.DataArray(rand_data, dims=['time', 'lat', 'lon'])
tempature

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.61645854, 281.28271168, 281.84226324, 278.61198526],
        [287.41265079, 284.35126492, 291.37705225, 282.52505946],
        [285.21029519, 282.25033567, 284.95447723, 286.34831043]],

       [[283.26057525, 289.01248113, 275.0626402 , 275.4539616 ],
        [284.68249285, 280.94575736, 280.49077327, 278.68237371],
        [282.10345323, 282.13403979, 275.69059717, 276.29680092]],

       [[283.45758266, 278.64363301, 277.70261278, 286.66049153],
        [286.64976732, 281.79493996, 293.54110883, 281.47250044],
        [287.67739165, 285.81404815, 290.57404849, 286.02542803]],

       [[277.12882683, 285.90567904, 290.64640203, 285.88070743],
        [283.22915585, 285.61280549, 276.44904728, 282.03546889],
        [278.45403384, 281.45890968, 288.01316258, 279.06508788]],

       [[287.02633878, 277.1492867 , 281.77224677, 277.13671493],
        [275.67401883, 283.77637672, 272.42099945, 281.2700804 ],
        [289.49935391, 271.0793694 , 275.71341183, 274.84101139]]])
Dimensions without coordinates: time, lat, lon

This has already been improved from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can associate arrays representing the values for the coordinates for each of these dimensions with the data when we create the `DataArray`. We'll see this in the next example.

# Section 2: Create a `DataArray` with named Coordinates

## Section 2.1: Make time and space coordinates

Here we will use [Pandas](https://foundations.projectpythia.org/core/pandas.html) to create an array of [datetime data](https://foundations.projectpythia.org/core/datetime.html), which we will then use to create a `DataArray` with a named coordinate `time`.

In [8]:
times_index = pd.date_range('2018-01-01', periods=5)
times_index

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [9]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

### Section 2.1.1: Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [10]:
tempature = xr.DataArray(rand_data, coords=[times_index, lats, lons], dims=['time', 'lat', 'lon'])
tempature

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.61645854, 281.28271168, 281.84226324, 278.61198526],
        [287.41265079, 284.35126492, 291.37705225, 282.52505946],
        [285.21029519, 282.25033567, 284.95447723, 286.34831043]],

       [[283.26057525, 289.01248113, 275.0626402 , 275.4539616 ],
        [284.68249285, 280.94575736, 280.49077327, 278.68237371],
        [282.10345323, 282.13403979, 275.69059717, 276.29680092]],

       [[283.45758266, 278.64363301, 277.70261278, 286.66049153],
        [286.64976732, 281.79493996, 293.54110883, 281.47250044],
        [287.67739165, 285.81404815, 290.57404849, 286.02542803]],

       [[277.12882683, 285.90567904, 290.64640203, 285.88070743],
        [283.22915585, 285.61280549, 276.44904728, 282.03546889],
        [278.45403384, 281.45890968, 288.01316258, 279.06508788]],

       [[287.02633878, 277.1492867 , 281.77224677, 277.13671493],
        [275.67401883, 283.77637672, 272.42099945, 281.2700804 ],
        [289.49935391, 271.0793694 , 275.71341183, 274.84101139]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

### Section 2.1.2: Set useful attributes

We can also set some attribute metadata, which will help provide clear descriptions of the data. In this case, we can specify that we're looking at 'air_temperature' data and the units are 'kelvin'.

In [11]:
tempature.attrs['units'] = 'kelvin'
tempature.attrs['standard_name'] = 'air_temperature'

tempature

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[290.61645854, 281.28271168, 281.84226324, 278.61198526],
        [287.41265079, 284.35126492, 291.37705225, 282.52505946],
        [285.21029519, 282.25033567, 284.95447723, 286.34831043]],

       [[283.26057525, 289.01248113, 275.0626402 , 275.4539616 ],
        [284.68249285, 280.94575736, 280.49077327, 278.68237371],
        [282.10345323, 282.13403979, 275.69059717, 276.29680092]],

       [[283.45758266, 278.64363301, 277.70261278, 286.66049153],
        [286.64976732, 281.79493996, 293.54110883, 281.47250044],
        [287.67739165, 285.81404815, 290.57404849, 286.02542803]],

       [[277.12882683, 285.90567904, 290.64640203, 285.88070743],
        [283.22915585, 285.61280549, 276.44904728, 282.03546889],
        [278.45403384, 281.45890968, 288.01316258, 279.06508788]],

       [[287.02633878, 277.1492867 , 281.77224677, 277.13671493],
        [275.67401883, 283.77637672, 272.42099945, 281.2700804 ],
        [289.49935391, 271.0793694 , 275.71341183, 274.84101139]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

### Section 2.1.3: Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:

In [12]:
tempature_in_celsius = tempature - 273.15
tempature_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[17.46645854,  8.13271168,  8.69226324,  5.46198526],
        [14.26265079, 11.20126492, 18.22705225,  9.37505946],
        [12.06029519,  9.10033567, 11.80447723, 13.19831043]],

       [[10.11057525, 15.86248113,  1.9126402 ,  2.3039616 ],
        [11.53249285,  7.79575736,  7.34077327,  5.53237371],
        [ 8.95345323,  8.98403979,  2.54059717,  3.14680092]],

       [[10.30758266,  5.49363301,  4.55261278, 13.51049153],
        [13.49976732,  8.64493996, 20.39110883,  8.32250044],
        [14.52739165, 12.66404815, 17.42404849, 12.87542803]],

       [[ 3.97882683, 12.75567904, 17.49640203, 12.73070743],
        [10.07915585, 12.46280549,  3.29904728,  8.88546889],
        [ 5.30403384,  8.30890968, 14.86316258,  5.91508788]],

       [[13.87633878,  3.9992867 ,  8.62224677,  3.98671493],
        [ 2.52401883, 10.62637672, -0.72900055,  8.1200804 ],
        [16.34935391, -2.0706306 ,  2.56341183,  1.69101139]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

# Section 3: The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`, which is a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (which we will practice in a later tutorial). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

## Section 3.1: Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [13]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times_index, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 994.64981513, 1007.61879674,  998.26208483,  994.27601692],
        [ 999.53343253, 1007.81954548,  997.99362313, 1001.49539435],
        [ 998.15980216,  998.58357058, 1000.85766347,  996.7604484 ]],

       [[1007.5487362 , 1004.28454608, 1000.13034576,  994.02713262],
        [1003.00556649, 1009.9595411 , 1001.66329776, 1001.89620608],
        [1005.05494705, 1009.7398974 ,  995.16532876,  998.66632311]],

       [[ 993.00167859, 1002.15304218,  999.79432648, 1004.54677196],
        [ 990.17976659, 1002.12775158,  992.2745104 , 1001.34779938],
        [1000.73905947, 1003.29387701, 1003.76118702, 1004.8795231 ]],

       [[1003.11731879, 1000.9669648 ,  996.21871162,  996.97799519],
        [ 998.42794045, 1000.17461964, 1001.42629789, 1007.79489392],
        [1002.85906276, 1000.6155788 , 1005.53540083,  991.27256497]],

       [[ 998.10452998, 1002.66988881,  998.46291552, 1002.23827684],
        [ 993.1490431 , 1001.65511165, 1001.96385318, 1007.36310539],
        [ 996.63182169,  994.85774074, 1001.03715137, 1000.00188318]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

## Section 3.2: Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [14]:
ds = xr.Dataset(data_vars={'Temperature': tempature, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 290.6 281.3 281.8 ... 275.7 274.8
    Pressure     (time, lat, lon) float64 994.6 1.008e+03 ... 1.001e+03 1e+03

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

## Section 3.3: Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [15]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 994.64981513, 1007.61879674,  998.26208483,  994.27601692],
        [ 999.53343253, 1007.81954548,  997.99362313, 1001.49539435],
        [ 998.15980216,  998.58357058, 1000.85766347,  996.7604484 ]],

       [[1007.5487362 , 1004.28454608, 1000.13034576,  994.02713262],
        [1003.00556649, 1009.9595411 , 1001.66329776, 1001.89620608],
        [1005.05494705, 1009.7398974 ,  995.16532876,  998.66632311]],

       [[ 993.00167859, 1002.15304218,  999.79432648, 1004.54677196],
        [ 990.17976659, 1002.12775158,  992.2745104 , 1001.34779938],
        [1000.73905947, 1003.29387701, 1003.76118702, 1004.8795231 ]],

       [[1003.11731879, 1000.9669648 ,  996.21871162,  996.97799519],
        [ 998.42794045, 1000.17461964, 1001.42629789, 1007.79489392],
        [1002.85906276, 1000.6155788 , 1005.53540083,  991.27256497]],

       [[ 998.10452998, 1002.66988881,  998.46291552, 1002.23827684],
        [ 993.1490431 , 1001.65511165, 1001.96385318, 1007.36310539],
        [ 996.63182169,  994.85774074, 1001.03715137, 1000.00188318]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [16]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 994.64981513, 1007.61879674,  998.26208483,  994.27601692],
        [ 999.53343253, 1007.81954548,  997.99362313, 1001.49539435],
        [ 998.15980216,  998.58357058, 1000.85766347,  996.7604484 ]],

       [[1007.5487362 , 1004.28454608, 1000.13034576,  994.02713262],
        [1003.00556649, 1009.9595411 , 1001.66329776, 1001.89620608],
        [1005.05494705, 1009.7398974 ,  995.16532876,  998.66632311]],

       [[ 993.00167859, 1002.15304218,  999.79432648, 1004.54677196],
        [ 990.17976659, 1002.12775158,  992.2745104 , 1001.34779938],
        [1000.73905947, 1003.29387701, 1003.76118702, 1004.8795231 ]],

       [[1003.11731879, 1000.9669648 ,  996.21871162,  996.97799519],
        [ 998.42794045, 1000.17461964, 1001.42629789, 1007.79489392],
        [1002.85906276, 1000.6155788 , 1005.53540083,  991.27256497]],

       [[ 998.10452998, 1002.66988881,  998.46291552, 1002.23827684],
        [ 993.1490431 , 1001.65511165, 1001.96385318, 1007.36310539],
        [ 996.63182169,  994.85774074, 1001.03715137, 1000.00188318]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files in later tutorials today.

# Summary

In our initial tutorial, we utilized the `DataArray` and `Dataset` objects to handle and analyze spatial data. We focused on organizing extensive global climate datasets and examining the fluctuations within Earth's climate system.